In [1]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [12]:
df = pl.read_csv(
    "../data/processed/df_procesada.csv",
    separator=";",
    columns=["ESTANCIA", "COD_HOSPITAL", "DIAGNOSTICO1", "ANIO_EGRESO"],
)

In [15]:
resumen_egresos_nacionales = df.group_by(
    pl.col(["ANIO_EGRESO", "COD_HOSPITAL", "DIAGNOSTICO1"])
).agg([pl.count().alias("n_egresos"), pl.col("ESTANCIA").sum().alias("dias_estada_totales")])